# My Update
In this notebook I will expand the functionalities of the current RAG system implemented together with my group.

In [1]:
import json
import pandas as pd

from rag_service.rag_chat import rag_interaction, ChatMemory

# CHAT
Use case as chat: make general queries on the Knowledge Base and KPI Generation.

## Knowledge Base
Instead of getting the Knowledge Base data through the API, as we did for the project, I downloaded the data so I could have it locally and not have to log in every time to get it. The data in the KB consists of two JSON files that contain the description of the available KPIs and the list of machines with the corresponding KPIs. I also added a txt file with general information about the factory site.

In [34]:
file_path = "data/documentation.txt"

with open(file_path, "r") as file:
    documentation = file.read()

In [3]:
file_path = "data/kpis.json"

with open(file_path, "r") as file:
    kpi_data = json.load(file)

In [4]:
file_path = "data/machines.json"

with open(file_path, "r") as file:
    machine_data = json.load(file)

In [5]:
data = [kpi_data, machine_data]
data.append({"type": "txt", "content": documentation}) # in this way we are able to include other file types to the kb

## Queries

In [6]:
query = "How many machines are there?"
response = rag_interaction(data, query)

Using chatbot mode...
Creating new vector index...


Parsing nodes:   0%|          | 0/3 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
print(response)

There are 16 machines in total.


---

In [8]:
query = "How many KPIs are there?"
response = rag_interaction(data, query)

Using chatbot mode...
Loading precomputed vector index...


In [9]:
print(response)

There are 14 key performance indicators (KPIs).


---

In [10]:
query = "Are there KPIs related to consumption?"
response = rag_interaction(data, query)

Using chatbot mode...
Loading precomputed vector index...


In [11]:
print(response)

Yes, there are KPIs related to consumption. These include "consumption", "consumption_working" and "consumption_idle".


---

In [12]:
query = "Can you speak italian?"
response = rag_interaction(data, query)

Using chatbot mode...
Loading precomputed vector index...


In [13]:
print(response)

No, I am not capable of speaking any language, including Italian. My purpose is to assist with generating text based on the information provided to me, but I do not have the ability to engage in verbal conversations or express myself through speech.


---

In [16]:
query = "Show me the list of the KPIs available for the Assembly Machine 1"
response = rag_interaction(data, query)

Using chatbot mode...
Loading precomputed vector index...


In [17]:
print(response)

The KPIs available for the Assembly Machine 1 are:
- working_time
- idle_time
- offline_time
- average_cycle_time
- consumption
- cost
- cost_idle
- good_cycles
- bad_cycles


---

In [18]:
query = "Generate a new KPI"
response = rag_interaction(data, query)

Using chatbot mode...
Loading precomputed vector index...


In [19]:
print(response)

{
  "KPIs": [
    {
      "name": "Total Idle Time",
      "type": "Consumption",
      "description": "The total idle time in minutes across all machines.",
      "unit_of_measure": "Minutes",
      "formula": "(consumption_idle / consumption) * 100"
    }
  ]
}


# REPORT
I use a different Knowledge Base for the Report Generation use case. This is justified by the fact that this RAG should be integrated with a proper interface, and the user should be able to choose whether to `CHAT` or to make a `REPORT`. I assume that the user can choose a period of time and a way to aggregate the data in the dataset (the pkl file), together with the machines he is interested in. The Knowledge Base would be made of the data the user is interested in, so it is better not to store the embeddings in this case. 

In the following section, I will create a JSON file that could be the product of the choices of the user.

## Report Data
In this section I am going to extract a small part of the orginal dataset provided by the professors. I will use this subset of data to test the report generation functionality of the RAG system.

I considered 2 machines: the Riveting Machine and the Laser Cutter, in the working week that goes from the 14/10/2024 to the 18/10/2024. I aggregated the data doing the average, obtaining 28 rows of data.

In [ ]:
df = pd.read_pickle('data/smart_app_data.pkl')
df.head()

,time,asset_id,name,kpi,sum,avg,min,max
0,2024-03-01T00:00:00Z,ast-yhccl1zjue2t,Large Capacity Cutting Machine 1,working_time,0.000000,0.000000,0.0,0.000000
1,2024-03-01T00:00:00Z,ast-yhccl1zjue2t,Large Capacity Cutting Machine 1,idle_time,0.000000,0.000000,0.0,0.000000
2,2024-03-01T00:00:00Z,ast-yhccl1zjue2t,Large Capacity Cutting Machine 1,offline_time,0.000000,0.000000,0.0,0.000000
3,2024-03-01T00:00:00Z,ast-yhccl1zjue2t,Large Capacity Cutting Machine 1,consumption,0.066106,0.002321,0.0,0.066106
4,2024-03-01T00:00:00Z,ast-yhccl1zjue2t,Large Capacity Cutting Machine 1,power,NaN,0.003673,0.0,0.012801


In [ ]:
# Convert the "time" column to just the date (YYYY-MM-DD) format
df['time'] = pd.to_datetime(df['time']).dt.date
df.head()

,time,asset_id,name,kpi,sum,avg,min,max
0,2024-03-01,ast-yhccl1zjue2t,Large Capacity Cutting Machine 1,working_time,0.000000,0.000000,0.0,0.000000
1,2024-03-01,ast-yhccl1zjue2t,Large Capacity Cutting Machine 1,idle_time,0.000000,0.000000,0.0,0.000000
2,2024-03-01,ast-yhccl1zjue2t,Large Capacity Cutting Machine 1,offline_time,0.000000,0.000000,0.0,0.000000
3,2024-03-01,ast-yhccl1zjue2t,Large Capacity Cutting Machine 1,consumption,0.066106,0.002321,0.0,0.066106
4,2024-03-01,ast-yhccl1zjue2t,Large Capacity Cutting Machine 1,power,NaN,0.003673,0.0,0.012801


In [ ]:
df['time'] = pd.to_datetime(df['time'])

In [ ]:
# Keep only the time, name, kpi, and avg columns
df = df[['time', 'name', 'kpi', 'avg']]
df.head()

,time,name,kpi,avg
0,2024-03-01,Large Capacity Cutting Machine 1,working_time,0.000000
1,2024-03-01,Large Capacity Cutting Machine 1,idle_time,0.000000
2,2024-03-01,Large Capacity Cutting Machine 1,offline_time,0.000000
3,2024-03-01,Large Capacity Cutting Machine 1,consumption,0.002321
4,2024-03-01,Large Capacity Cutting Machine 1,power,0.003673


In [ ]:
# Consider only the working week that goes from the 2024-10-14 to the 2024-10-18
df_week = df[(df['time'] >= '2024-10-14') & (df['time'] <= '2024-10-18')]
df_week.head()

,time,name,kpi,avg
92482,2024-10-14,Large Capacity Cutting Machine 1,working_time,25406.000000
92483,2024-10-14,Large Capacity Cutting Machine 1,idle_time,21791.000000
92484,2024-10-14,Large Capacity Cutting Machine 1,offline_time,888.000000
92485,2024-10-14,Large Capacity Cutting Machine 1,consumption,0.004759
92486,2024-10-14,Large Capacity Cutting Machine 1,power,0.004834


In [ ]:
# Keep only two machines: Riveting Machine and Laser Cutter
df_week = df_week[(df_week['name'] == 'Riveting Machine') | (df_week['name'] == 'Laser Cutter')]
df_week.head()

,time,name,kpi,avg
92768,2024-10-14,Riveting Machine,working_time,30038.000000
92769,2024-10-14,Riveting Machine,idle_time,17380.000000
92770,2024-10-14,Riveting Machine,offline_time,2698.000000
92771,2024-10-14,Riveting Machine,consumption,0.001000
92772,2024-10-14,Riveting Machine,power,0.000131


In [ ]:
# Aggregate the data by machine and kpi and consider the average over the week
df_week = df_week.groupby(['name', 'kpi']).mean()
df_week.reset_index(inplace=True)
df_week = df_week.drop(columns=['time'])
df_week

,name,kpi,avg
0,Laser Cutter,average_cycle_time,9.511204
1,Laser Cutter,bad_cycles,2.200000
2,Laser Cutter,consumption,0.000000
3,Laser Cutter,consumption_idle,0.000000
4,Laser Cutter,consumption_working,0.000000
5,Laser Cutter,cost,0.000000
6,Laser Cutter,cost_idle,0.000000
7,Laser Cutter,cost_working,0.000000
8,Laser Cutter,cycles,1.000000
9,Laser Cutter,good_cycles,553.600000


In [ ]:
# count how many rows of df_week have avg column equal to 0
len(df_week[df_week['avg'] == 0])

7

In [ ]:
# Convert DataFrame to JSON
json_result = df_week.to_json(orient="records", indent=4)  

# Parse the JSON string back into a Python list
list_result = json.loads(json_result)

# Add metadata to the JSON result
result = {
    "metadata": {
        "working_week": "2024-10-14 to 2024-10-18",
        "description": "The average values are calculated over the days within the specified week."
    },
    "data": list_result
}

#### Save JSON to a file

In [ ]:
"""# Save JSON to a file
with open("data/report_data.json", "w") as f:
    json.dump(result, f, indent=4)"""

## Report Generation

In [20]:
file_path = "data/report_data.json"

with open(file_path, "r") as file:
    report_data = json.load(file)

In [21]:
data = []
data.append({"type": "json", "content": report_data})

In [22]:
response = rag_interaction(data, generate_report=True)

Generating report with dynamic KB...


Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

In [23]:
print(response)

**Industrial Machine Performance Report**

**Time Period Covered:** October 14, 2024 - October 18, 2024

This report provides an in-depth analysis of the performance of two industrial machines: Laser Cutter and Riveting Machine. The analysis is based on key performance indicators (KPIs) such as average cycle time, consumption, cost, idle time, offline time, power usage, working time, good cycles, bad cycles, and cycles.

**Machine Performance Comparison**

| KPI | Laser Cutter | Riveting Machine |
| --- | --- | --- |
| Average Cycle Time | 9.51 seconds (avg) | 3.31 seconds (avg) |
| Consumption | 0.00 units (low) | 0.001 units (low) |
| Cost | 0.00 units (zero) | 0.001 units (low) |
| Idle Time | 19,150.2 hours (high) | 26,285.8 hours (high) |
| Offline Time | 327.2 hours (moderate) | 703.0 hours (moderate) |
| Power Usage | 0.00012 units (very low) | 0.00012 units (very low) |
| Working Time | 7,144.9 hours (high) | 4,715.8 hours (moderate) |

**Key Insights and Recommendations**

* T

---

### Let's visualize the response!

---

**Industrial Machine Performance Report**

**Time Period Covered:** October 14, 2024 - October 18, 2024

This report provides an in-depth analysis of the performance of two industrial machines: Laser Cutter and Riveting Machine. The analysis is based on key performance indicators (KPIs) such as average cycle time, consumption, cost, idle time, offline time, power usage, working time, good cycles, bad cycles, and cycles.

**Machine Performance Comparison**

| KPI | Laser Cutter | Riveting Machine |
| --- | --- | --- |
| Average Cycle Time | 9.51 seconds (avg) | 3.31 seconds (avg) |
| Consumption | 0.00 units (low) | 0.001 units (low) |
| Cost | 0.00 units (zero) | 0.001 units (low) |
| Idle Time | 19,150.2 hours (high) | 26,285.8 hours (high) |
| Offline Time | 327.2 hours (moderate) | 703.0 hours (moderate) |
| Power Usage | 0.00012 units (very low) | 0.00012 units (very low) |
| Working Time | 7,144.9 hours (high) | 4,715.8 hours (moderate) |

**Key Insights and Recommendations**

* The Laser Cutter has a significantly shorter average cycle time compared to the Riveting Machine, indicating potential for process optimization.
* Both machines have very low consumption rates, suggesting efficient energy usage.
* However, the Riveting Machine has a lower idle time, which may indicate a more productive work schedule.
* The offline and working times suggest varying levels of downtime and productivity across both machines.
* Power usage is extremely low in both cases, indicating efficient operation.

**Actionable Recommendations**

1. **Optimize Process**: Analyze and optimize the process to reduce average cycle time for the Laser Cutter.
2. **Monitor Energy Consumption**: Regularly monitor energy consumption to ensure it remains at optimal levels.
3. **Improve Productivity**: Investigate ways to increase productivity of the Riveting Machine, particularly during working times.

This report provides a comprehensive analysis of machine performance and offers actionable recommendations to improve efficiency, reduce waste, and optimize resource allocation.

# MEMORY
Exploit the memory functionality.

### Test

In [38]:
# Use the kb of the chat use case
data = [kpi_data, machine_data]
data.append({"type": "txt", "content": documentation})

In [39]:
memory = ChatMemory()

query = "Are there metal cutter machines?"
response = rag_interaction(data, query, memory=memory)
print(response)

Using chatbot mode...
Loading precomputed vector index...
Yes, there are metal cutter machines. They consist of six individual machines within the company's single site, each classified under this category.


In [40]:
query = "What are their names?"
response = rag_interaction(data, query, memory=memory)
print(response)

Using chatbot mode...
Loading precomputed vector index...
The metal cutter machines have the following names:
1. Large Capacity Cutting Machine 1
2. Medium Capacity Cutting Machine 1
3. Low Capacity Cutting Machine 1
4. Medium Capacity Cutting Machine 2
5. Large Capacity Cutting Machine 2
6. Medium Capacity Cutting Machine 3


### Error Correction
Handle cases where users correct themselves mid-dialogue.

In [10]:
# Use the kb of the chat use case
data = [kpi_data, machine_data]
data.append({"type": "txt", "content": documentation})

memory = ChatMemory()

In [11]:
query = "What are the KPIs of the plasma cutter machine?"
response = rag_interaction(data, query, memory=memory)
print(response)

Using chatbot mode...
Loading precomputed vector index...
I'm unable to provide information on a "plasma cutter machine" as it does not appear to be one of the specified machine types in the provided context. The context only mentions specific types of machines such as metal cutters, riveters, laser cutters, assemblers, welding machines, and testing machines. If you could provide more information or clarify which type of machine you are referring to, I'd be happy to try and assist you further.


In [12]:
query = "Sorry, I meant the laser cutter machine."
response = rag_interaction(data, query, memory=memory)
print(response)

Using chatbot mode...
Loading precomputed vector index...
The KPIs for a Laser Cutter machine are:

- working_time
- idle_time
- offline_time
- average_cycle_time
- consumption
- power
- cycles


---

In [13]:
# Use the kb of the chat use case
data = [kpi_data, machine_data]
data.append({"type": "txt", "content": documentation})

memory = ChatMemory()

In [14]:
query = "How many plasma cutter machines are there?"
response = rag_interaction(data, query, memory=memory)
print(response)

Using chatbot mode...
Loading precomputed vector index...
I'm unable to provide an answer to that question as it seems to be outside of the scope of the provided knowledge base.


In [15]:
query = "Sorry, I meant the laser cutter machine."
response = rag_interaction(data, query, memory=memory)
print(response)

Using chatbot mode...
Loading precomputed vector index...
There is only one machine classified as a "Laser Cutter".


In [16]:
query = "And for the metal cutters?"
response = rag_interaction(data, query, memory=memory)
print(response)

Using chatbot mode...
Loading precomputed vector index...
There are six metal cutter machines.


In [18]:
query = "And assemblers?"
response = rag_interaction(data, query, memory=memory)
print(response)

Using chatbot mode...
Loading precomputed vector index...
There are three assembler machines.


In [19]:
query = "And welding?"
response = rag_interaction(data, query, memory=memory)
print(response)

Using chatbot mode...
Loading precomputed vector index...
There are two machines classified as welding machines.


In [20]:
query = "And testing?"
response = rag_interaction(data, query, memory=memory)
print(response)

Using chatbot mode...
Loading precomputed vector index...
There are three machines classified as "Testing Machines".


In [21]:
query = "And drills?"
response = rag_interaction(data, query, memory=memory)
print(response)

Using chatbot mode...
Loading precomputed vector index...
There is no mention of 'drills' in the provided context information. I'm unable to provide an answer as the topic is not present in the given data.


### Personalized Recommendations

In [35]:
# Use the kb of the chat use case
data = [kpi_data, machine_data]
data.append({"type": "txt", "content": documentation})

memory = ChatMemory()

In [36]:
query = "Can you tell me about the efficiency KPIs of the machines?"
response = rag_interaction(data, query, memory=memory)
print(response)

Using chatbot mode...
Loading precomputed vector index...
The company has a variety of machine types that are monitored using different sets of key performance indicators (KPIs). There are three distinct categories of KPIs: Time, Consumption, Efficiency, Cost, and Quality. However, the Efficiency category is further sub-divided into two main groups - Power and Cycles.

In terms of specific KPIs, the Efficiency group includes metrics related to power consumption as a percentage and cycle counts for machines. These metrics provide insights into how efficiently each machine type is operating in relation to its power usage and ability to complete cycles.


In [37]:
query = "Can you generate a new KPI out of them?"
response = rag_interaction(data, query, memory=memory)
print(response)

Using chatbot mode...
Loading precomputed vector index...
{
  "KPIs": [
    {
      "name": "energy_efficiency",
      "type": "Efficiency",
      "description": "Energy efficiency ratio of the machine",
      "unit_of_measure": "",
      "formula": "(power / (consumption_working + consumption_idle)) * 100"
    }
  ]
}
